<a href="https://colab.research.google.com/github/ambicapvs/ambica_info5731_spring2021/blob/main/SEC203_G5projectfirstdeliverable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Collection**

In [26]:
#Gathering data and Saving to CSV File. 
import os
import tweepy as tw
import pandas as pd
import csv

accesstoken = "1363750390383013890-kmomosFdwL8bOz6BKFcAXE6dlpFFXC"
access_secret = "***REDACTED BECAUSE PERSONAL ACCOUNT SECRET***"
consumerkey = "egBVddzEfnyK3mY9OcfDMLhdg"
consumersecret = "***REDACTED BECAUSE PERSONAL ACCOUNT SECRET***"
auth = tw.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesstoken, access_secret)

def getTweets(searchword):
  api = tw.API(auth, wait_on_rate_limit=True)
  date_since = "2020-12-01"
  print("Gathering tweets for Vaccine "+searchword)
  tweets = tw.Cursor(api.search, q=searchword, count= 1000, lang="en", since=date_since).items(10000)
  return list(tweets)

        
def main():
  #vaccines = ['moderna', 'pfizer', 'johnsonandjohnson'] # 'jjvaccine','jandjvaccine']
  modernalist = getTweets('moderna -filter:retweets')
  pfizerlist = getTweets('pfizer -filter:retweets')
  with open("vaccinetweets.csv","w") as csv_file:
    print("Writing CSV File")
    fieldnames = ['tweetid', 'moderna', 'pfizer']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(10000):
      writer.writerow({'tweetid': i , 'moderna': str(modernalist[i].text), 'pfizer': str(pfizerlist[i].text)})  #, 'johnsonandjohnson': str(johnsonandjohnsonlist[i].text)})


if __name__ == "__main__":
  main()




Gathering tweets for Vaccine moderna
Gathering tweets for Vaccine pfizer
Writing CSV File


**CSV to dataframe**

In [27]:

dataframe = pd.read_csv('/content/vaccinetweets.csv',sep=',',engine ='python', header=0)
dataframe

,tweetid,moderna,pfizer
0,0,2nd Moderna shot yesterday and had a 13 hour d...,@mredm99 Yes shipments into the ontario hub an...
1,1,@dysthymikey @scuriiosa 10000% esp if it’s Mod...,@catameep Not nearly as good as the 90+% effic...
2,2,RT @yungbiryanii: dj: all the moderna mamis an...,RT @JuddLegum: 3. Corporations that backed the...
3,3,RT @TurnbullWhitby: What Ford has said is not ...,New availability: Centura\nDicks Sporting Good...
4,4,@RosieBarton @ccouja @yelich_ivana More than h...,RT @objsucks: the two pfizer doses entering my...
...,...,...,...
9995,9995,RT @yungbiryanii: dj: all the moderna mamis an...,RT @ummmkiara: csub is doing pfizer vaccine wa...
9996,9996,RT @knomia: Got my 1st moderna shot yesterday ...,@Sloths7777 Got Pfizer 2nd dose today &amp; ye...
9997,9997,RT @yungbiryanii: dj: all the moderna mamis an...,RT @MarkLevineNYC: This is shaping up to be on...
9998,9998,RT @RoyCecely: There is certainly still volati...,RT @yungbiryanii: dj: all the moderna mamis an...


**Data cleaning**

In [28]:
import requests
from textblob import Word
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from textblob import TextBlob
from nltk.stem import PorterStemmer


def CleanData(dfcolumn):
  linksremoved = dfcolumn.replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)
  punctuationremoved = linksremoved.str.replace('[^\w\s]','')
  spclcharsremoved = punctuationremoved.str.replace('[^A-Za-z0-9 ]+','')
  numsremoved = spclcharsremoved.str.replace('[^A-Za-z ]+','')
  data = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
  a = data.text
  stop = a.split()
  stopwordsremoved = numsremoved.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  lowercased = stopwordsremoved.apply(lambda x: " ".join(x.lower() for x in x.split()))
  tokenized = lowercased.apply(lambda x: TextBlob(x).words)
  st = PorterStemmer()
  stemmed = tokenized.apply(lambda x: " ".join([st.stem(word) for word in x]))
  #Lemmatization
  lemmatized = stemmed.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  return lemmatized

modn = CleanData(dataframe['moderna'])
pfizr = CleanData(dataframe['pfizer'])
dataframe['moderna_cleaned'] = modn
dataframe['pfizer_cleaned'] = pfizr

dataframe


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,tweetid,moderna,pfizer,moderna_cleaned,pfizer_cleaned
0,0,2nd Moderna shot yesterday and had a 13 hour d...,@mredm99 Yes shipments into the ontario hub an...,nd moderna shot yesterday hour day today works...,mredm ye shipment ontario hub also undergo kin...
1,1,@dysthymikey @scuriiosa 10000% esp if it’s Mod...,@catameep Not nearly as good as the 90+% effic...,dysthymikey scuriiosa esp moderna,catameep not nearli good efficaci moderna pfiz...
2,2,RT @yungbiryanii: dj: all the moderna mamis an...,RT @JuddLegum: 3. Corporations that backed the...,rt yungbiryanii dj moderna mami pfizer papi le...,rt juddlegum corpor back sponsor georgia voter...
3,3,RT @TurnbullWhitby: What Ford has said is not ...,New availability: Centura\nDicks Sporting Good...,rt turnbullwhitbi what ford said accur fed gov...,new avail centuradick sport good park lot h pf...
4,4,@RosieBarton @ccouja @yelich_ivana More than h...,RT @objsucks: the two pfizer doses entering my...,rosiebarton ccouja yelichivana more half milli...,rt objsuck two pfizer dose enter bloodstream h...
...,...,...,...,...,...
9995,9995,RT @yungbiryanii: dj: all the moderna mamis an...,RT @ummmkiara: csub is doing pfizer vaccine wa...,rt yungbiryanii dj moderna mami pfizer papi le...,rt ummmkiara csub pfizer vaccin walkin today a...
9996,9996,RT @knomia: Got my 1st moderna shot yesterday ...,@Sloths7777 Got Pfizer 2nd dose today &amp; ye...,rt knomia got st moderna shot yesterday work s...,sloth got pfizer nd dose today amp ye i contin...
9997,9997,RT @yungbiryanii: dj: all the moderna mamis an...,RT @MarkLevineNYC: This is shaping up to be on...,rt yungbiryanii dj moderna mami pfizer papi le...,rt marklevinenyc thi shape one biggest day vax...
9998,9998,RT @RoyCecely: There is certainly still volati...,RT @yungbiryanii: dj: all the moderna mamis an...,rt roycec there certainli still volatil global...,rt yungbiryanii dj moderna mami pfizer papi le...
